In [174]:
import numpy as np
import matplotlib.pyplot as plt
from tkinter import *
import time

In [40]:
class Vector2:
    def __init__(self, x: float, y: float):
        self.x = x
        self.y = y

    def __str__(self):
        return f"({self.x}, {self.y})"
    
    def __add__(self, other):
        if isinstance(other, Vector2):
            return Vector2(self.x+other.x, self.y+other.y)
        elif isinstance(other, (int, float)):
            return Vector2(self.x+other, self.y+other)
        else:
            raise TypeError("Unsupported operand type for +")

    def __sub__(self, other):
        if isinstance(other, Vector2):
            return Vector2(self.x-other.x, self.y-other.y)
        elif isinstance(other, (int, float)):
            return Vector2(self.x-other, self.y-other)
        else:
            raise TypeError("Unsupported operand type for -")

    def __mul__(self, other):
        if isinstance(other, Vector2):
            return Vector2(self.x*other.x, self.y*other.y)
        elif isinstance(other, (int, float)):
            return Vector2(self.x*other, self.y*other)
        else:
            raise TypeError("Unsupported operand type for *")

    def __rmul__(self, scalar):
        return self.__mul__(scalar)

    def dot_product(self, other: Vector2):
        return (self.x * other.x) + (self.y * other.y)

    def cross_product(self, other: Vector2):
        return (self.x*other.y)-(self.y*other.x)

    def magnitude(self):
        return (self.x**2 + self.y**2) **0.5

In [257]:
class Planet:
    def __init__(self, mass: float, initial_speed: Vector2, initial_location: Vector2):
        self.mass = mass
        self.speed = initial_speed
        self.position = initial_location
        self.initial_speed = initial_speed
        self.initial_position = initial_location
        self.position_weight = self.position*mass

    def Precess(self, force: Vector2, time: float):
        acceleration = force*(1/self.mass)
        self.speed += acceleration * time
        self.position += self.speed * time


In [36]:
def calculate_gravitational_force(planet1: Planet, planet2: Planet):
    G = 6.674e-11
    distance_vector = planet2.position - planet1.position
    distance = distance_vector.magnitude()

    force_magnitude = (G*planet1.mass*planet2.mass)/(distance**2)
    force_direction = distance_vector*(1/distance)
    return force_magnitude*force_direction

In [188]:
sun_pos = Vector2(0,0)
sun_vel = Vector2(0,0)
sun = Planet(mass = 2e30, initial_speed = sun_vel, initial_location = sun_pos) 

earth_pos = Vector2(151e6, 0)

v0 = (G*sun.mass/earth_pos.x)**0.5 # Apprx orbital velocity based on 

earth_vel = Vector2(0, v0)
earth = Planet(mass = 2e30, initial_speed = earth_vel, initial_location = earth_pos)

time_interval = 0.1

for _ in range(0, 1000):
    start = time.time()
    force_earth_sun = calculate_gravitational_force(earth, sun)
    force_sun_earth = calculate_gravitational_force(sun, earth)

    earth.Precess(force_earth_sun, time_interval)
    sun.Precess(force_sun_earth, time_interval)
    end = time.time()
    time_interval = end-start

In [320]:
def drawPlanet(canvas, coords: Vector2, name):
    canvas.delete(f'{name}')
    x0, y0, x1, y1 = coords.x-2.5, coords.y-2.5, coords.x+2.5, coords.y+2.5
    canvas.create_oval(x0, y0, x1, y1, outline='black', fill='blue', tags=f'{name}')

def drawLine(canvas, old_coords, new_coords):
    canvas.create_line(old_coords.x, old_coords.y, new_coords.x, new_coords.y, fill='red', width=2)

def calc_planet_positions(p1:Planet, p2:Planet, time_interval):
    force_p1_p2 = calculate_gravitational_force(p1, p2)
    force_p2_p1 = calculate_gravitational_force(p2, p1)

    p1.Precess(force_p1_p2, time_interval)
    p2.Precess(force_p2_p1, time_interval)

def planetCoordsToCOM(p1: Planet, p2: Planet):
    x_com = ((p1.mass*p1.position.x)+(p2.mass*p2.position.x))/(p1.mass+p2.mass)
    y_com = ((p1.mass*p1.position.y)+(p2.mass*p2.position.y))/(p1.mass+p2.mass)

    com = Vector2(x_com, y_com)

    rel_pos1 = p1.position-com
    rel_pos2 = p2.position-com

    return rel_pos1, rel_pos2


def planetsToCanvasCoords(p1: Planet, p2: Planet):
    p1_com, p2_com = planetCoordsToCOM(p1, p2)
    larger_val = max(abs(p1_com.magnitude()), abs(p2_com.magnitude()))
    max_val = larger_val*1.1
    
    p1_ratio = (p1.position*(1/max_val))
    p2_ratio = (p2.position*(1/max_val))
    
    p1_coords = (500*p1_ratio)
    p2_coords = (500*p2_ratio)

    return p1_coords, p2_coords


def precessPlanets(canvas, planet1: Planet, planet2: Planet):
    #p1_prev_pos, p2_prev_pos = planetsToCanvasCoords(planet1, planet2)
    calc_planet_positions(planet1, planet2, 0.01)
    p1_coords, p2_coords = planetsToCanvasCoords(planet1, planet2)
    drawPlanet(canvas, p1_coords, 'p1')
    #drawLine(canvas, p1_prev_pos, p1_coords)
    drawPlanet(canvas, p2_coords, 'p2')
    #drawLine(canvas, p2_prev_pos, p2_coords)
    canvas.after(1, precessPlanets, canvas, planet1, planet2)

root = Tk()
root.title("Simulation")

sun_pos = Vector2(0,0)
sun_vel = Vector2(0,0)
sun = Planet(mass = 2e30, initial_speed = sun_vel, initial_location = sun_pos) 

earth_pos = Vector2(151e6, 0)

v0 = (G*sun.mass/earth_pos.x)**0.5 # Apprx orbital velocity based on 

earth_vel = Vector2(0, v0)
earth = Planet(mass = 2e30, initial_speed = earth_vel, initial_location = earth_pos)

#canvas
canvas = Canvas(root, width = 1000, height = 1000, bg='white')
canvas.pack()

precessPlanets(canvas, sun, earth)

root.mainloop()

In [326]:
sun_pos = Vector2(0,0)
sun_vel = Vector2(0,0)
sun = Planet(mass = 2e30, initial_speed = sun_vel, initial_location = sun_pos) 

earth_pos = Vector2(151e6, 0)

v0 = (G*sun.mass/earth_pos.x)**0.5 # Apprx orbital velocity based on 

earth_vel = Vector2(0, v0)
earth = Planet(mass = 2e30, initial_speed = earth_vel, initial_location = earth_pos)

a, b = planetCoordsToCOM(sun ,earth)
c, d = planetsToCanvasCoords2(a,b)

83050000.0
p1_rat:(-0.9090909090909091, 0.0)
p2_rat:(0.9090909090909091, 0.0)
p1_coords:(-1409.090909090909, -500.0)
p2_coords:(409.090909090909, -500.0)


In [325]:
def planetsToCanvasCoords2(p1: Vector2, p2: Vector2):
    #p1_com, p2_com = planetCoordsToCOM(p1, p2)
    larger_val = max(abs(p1.magnitude()), abs(p2.magnitude()))
    max_val = larger_val*1.1
    print(max_val)
    
    p1_ratio = (p1*(1/max_val))
    print(f'p1_rat:{p1_ratio}')
    p2_ratio = (p2*(1/max_val))
    print(f'p2_rat:{p2_ratio}')
    
    p1_coords = (1000*p1_ratio)-500
    print(f'p1_coords:{p1_coords}')
    p2_coords = (1000*p2_ratio)-500
    print(f'p2_coords:{p2_coords}')
    
    return p1_coords, p2_coords

In [306]:
print(a)
print(b)

(-64005641.70045313, -27723878.543134626)
(64005641.70045312, 27723878.54313463)


In [307]:
print(c)
print(d)

(82.90097603453023, 319.3346340468481)
(917.0990239654698, 680.6653659531519)
